In [ ]:
import cv2
import numpy as np
# Step 1: Load the image
image = cv2.imread('puppy_dog.jpg')
# Check if the image was successfully loaded
if image is None:
    print("Error: Could not load image.")
else:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0
    markers = cv2.watershed(image, markers)
    image[markers == -1] = [0, 0, 255] # Mark boundaries with red
    markers_display = markers.astype(np.uint8)
    markers_display = cv2.normalize(markers_display, None, 0, 255, cv2.NORM_MINMAX)
    cv2.imshow('Original Image', image)
    cv2.imshow('Binary Image', binary)
    cv2.imshow('Markers', markers_display)
    cv2.imshow('Segmented Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()